In [219]:
import pandas as pd
import numpy as np
import math
from functools import partial, reduce
import os
import re

In [220]:
df = pd.read_csv('C:/Users/lzoeckler/Desktop/4plex/allison_input/allison_quansys.csv')
df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
df['well_group'] = df['well_group'].fillna(method='ffill')
df.rename({'well_group': 'patient_id'}, axis=1, inplace=True)
df.drop(['some ID', 'statistic'], axis=1, inplace=True)
df.head()

,patient_id,well,error,HRP2_pg_ml,LDH_Pan_pg_ml,LDH_Pv_pg_ml,LDH_Pf_pg_ml,CRP_ng_ml
0,ah02_lyo sample wb (neat),c2,NaN,3.55,< 18.61,< 5.01,< 6.90,< 25.86
1,ah02_lyo sample wb 20x (1:20),d2,NaN,< 13.80,< 372.20,< 100.20,< 138.00,< 517.20
2,ah1_lyo sample wb (neat),c6,NaN,89.61,< 18.61,< 5.01,< 6.90,< 25.86
3,ah1_lyo sample wb 20x (1:20),d6,NaN,< 13.80,< 372.20,< 100.20,< 138.00,< 517.20
4,al10_f sample wb (neat),g4,NaN,< 0.69,< 18.61,< 5.01,< 6.90,< 25.86


In [221]:
def fix_concentrations(df):
    con = df['concentration'].partition(':')[2]
    con = con.partition(')')[0]
    if len(con) != 0:
        return con
    else:
        return '1'

In [222]:
def bad_func(x):
    return '_'.join(x.split(' ')[:-1]).replace('_20x', '').replace('_400x', '').replace('__', '')

In [223]:
samples_data = df.copy(deep=True)
samples_data['concentration'] = samples_data['patient_id'].apply(lambda x: x.partition(' ')[-1])
samples_data['patient_id'] = samples_data['patient_id'].apply(bad_func)
samples_data['concentration'] = samples_data.apply(fix_concentrations, axis=1)
samples_data = samples_data.sort_values(['patient_id', 'concentration'])
samples_data.head()

,patient_id,well,error,HRP2_pg_ml,LDH_Pan_pg_ml,LDH_Pv_pg_ml,LDH_Pf_pg_ml,CRP_ng_ml,concentration
0,ah02_lyo_sample_wb,c2,NaN,3.55,< 18.61,< 5.01,< 6.90,< 25.86,1
1,ah02_lyo_sample_wb,d2,NaN,< 13.80,< 372.20,< 100.20,< 138.00,< 517.20,20
2,ah1_lyo_sample_wb,c6,NaN,89.61,< 18.61,< 5.01,< 6.90,< 25.86,1
3,ah1_lyo_sample_wb,d6,NaN,< 13.80,< 372.20,< 100.20,< 138.00,< 517.20,20
4,al10_f_sample_wb,g4,NaN,< 0.69,< 18.61,< 5.01,< 6.90,< 25.86,1


In [224]:
samples_data.tail()

,patient_id,well,error,HRP2_pg_ml,LDH_Pan_pg_ml,LDH_Pv_pg_ml,LDH_Pf_pg_ml,CRP_ng_ml,concentration
61,sn_4_sample_wb,g9,NaN,< 13.80,494.55,317.73,< 138.00,< 517.20,20
64,sn_4_sample_wb,h9,NaN,< 276.00,< 7444.00,< 2004.00,< 2760.00,< 10344.00,400
57,sn_6_sample_wb,e10,NaN,< 0.69,< 18.61,11.34,< 6.90,< 25.86,1
62,sn_6_sample_wb,g10,NaN,< 13.80,745.8,503.45,< 138.00,< 517.20,20
65,sn_6_sample_wb,h10,NaN,< 276.00,< 7444.00,< 2004.00,< 2760.00,< 10344.00,400


In [225]:
samples_data.loc[samples_data['patient_id'].str.contains('pellet')].head()

,patient_id,well,error,HRP2_pg_ml,LDH_Pan_pg_ml,LDH_Pv_pg_ml,LDH_Pf_pg_ml,CRP_ng_ml,concentration
43,pellet_21_sample_wb,c12,NaN,< 0.69,> 58369.00,> 11372.00,29.86,< 25.86,1
51,pellet_21_sample_wb,d12,NaN,< 13.80,27847.86,13918.3,194.41,< 517.20,20
53,pellet_21_sample_wb,f12,NaN,< 276.00,< 7444.00,< 2004.00,< 2760.00,< 10344.00,400
44,pellet_26_sample_wb,c11,NaN,< 0.69,> 58369.00,> 11372.00,23.43,< 25.86,1
52,pellet_26_sample_wb,d11,NaN,< 13.80,41526.44,18288.69,< 138.00,< 517.20,20


In [226]:
samples_data.loc[samples_data['patient_id'].str.contains('pellet')].tail()

,patient_id,well,error,HRP2_pg_ml,LDH_Pan_pg_ml,LDH_Pv_pg_ml,LDH_Pf_pg_ml,CRP_ng_ml,concentration
46,pellet_4_sample_wb,d9,NaN,< 13.80,32845.23,15477.56,< 138.00,< 517.20,20
49,pellet_4_sample_wb,f9,NaN,< 276.00,< 7444.00,< 2004.00,< 2760.00,< 10344.00,400
41,pellet_6_sample_wb,c10,NaN,< 0.69,> 58369.00,> 11372.00,15.77,< 25.86,1
47,pellet_6_sample_wb,d10,NaN,< 13.80,37155.31,17097.73,< 138.00,< 517.20,20
50,pellet_6_sample_wb,f10,NaN,< 276.00,< 7444.00,< 2004.00,< 2760.00,< 10344.00,400


In [227]:
samples_data['concentration'].unique()

array(['1', '20', '400'], dtype=object)

In [228]:
sample_ids = samples_data['patient_id'].unique().tolist()
sample_set = set(sample_ids)

In [229]:
# threshhold values for various analytes
threshholds = {'HRP2_pg_ml': 2464, 'LDH_Pan_pg_ml': 58369,
               'LDH_Pv_pg_ml': 11372, 'LDH_Pf_pg_ml': 24164,
               'CRP_ng_ml': 35411}

In [230]:
# constant to apply to the threshhold for different dilutions
dil_constants = {'20': 1, '400': 20}

In [231]:
# dilution sets for various dilutions
dilution_sets = {'20': ('1', '20', 'fail'),
                 '400': ('20', '400', 'fail')} 

In [232]:
def return_decisions(low, high, fail='fail'):
    # Columns = [neat_above, neat_below, neat_LLQ, neat_ULQ, NA]
    # Rows = [dil_above, dil_below, dil_LLQ, dil_ULQ, NA]
    matrix = np.array([[high, high, high, high, fail],
                       [low, low, low, low, low],
                       [low, low, low, fail, low],
                       [high, high, fail, high, fail],
                       [high, fail, fail, high, fail]])
    # decisions for various analytes
    decisions = {'HRP2_pg_ml': matrix, 'LDH_Pan_pg_ml': matrix,
                 'LDH_Pv_pg_ml': matrix, 'LDH_Pf_pg_ml': matrix,
                 'CRP_ng_ml': matrix}
    return(decisions)

In [233]:
def run_compare(df, analyte_val, dil_val):
    above, below, LLQ, ULQ, NA = False, False, False, False, False
    val = df[analyte_val]
    thresh_val = dil_constants[dil_val] * threshholds[analyte_val]
    try:
        float_val = float(val)
        if math.isnan(float_val):
            NA = True
        elif float_val > thresh_val:
            above = True
        elif float_val < thresh_val:
            below = True
    except ValueError:
        if '<' in val:
            LLQ = True
        elif '>' in val:
            ULQ = True
    finally:
        return(np.array([above, below, LLQ, ULQ, NA]))

In [234]:
samples_data.loc[samples_data['patient_id'] == 'ah02_lyo_sample_wb']

,patient_id,well,error,HRP2_pg_ml,LDH_Pan_pg_ml,LDH_Pv_pg_ml,LDH_Pf_pg_ml,CRP_ng_ml,concentration
0,ah02_lyo_sample_wb,c2,NaN,3.55,< 18.61,< 5.01,< 6.90,< 25.86,1
1,ah02_lyo_sample_wb,d2,NaN,< 13.80,< 372.20,< 100.20,< 138.00,< 517.20,20


In [235]:
analyte_dfs = []
error_pids = {}
for analyte in threshholds.keys():
# for analyte in ['HRP2_pg_ml']:
    print(analyte)
    patient_dfs = []
    for pid in samples_data['patient_id'].unique():
#     for pid in ['	pellet_2_sample_wb']:
        patient_data = samples_data.loc[samples_data['patient_id'] == pid]
        dilution_values = sorted([val for val in patient_data['concentration'].unique() if val != '1'], key=len)
        best_decision = '1'
        for max_dilution in dilution_values:
#         for max_dilution in ['400']:
            dil_data = patient_data.loc[patient_data['concentration'].isin([best_decision, max_dilution])]          
            partial_compare = partial(run_compare, analyte_val=analyte, dil_val=max_dilution)
            dil_data['decision_vector'] = dil_data.apply(partial_compare, axis=1)
            decisions = return_decisions(best_decision, max_dilution)
            decision_matrix = decisions[analyte]
            best_df = pd.DataFrame(columns=['patient_id', 'errors', analyte,
                                            '{}_dilution'.format(analyte),
                                            '{}_well'.format(analyte),
                                            '{}_max_dilution'.format(analyte)])
            vector_low = dil_data.loc[dil_data['concentration'] == best_decision,
                                      'decision_vector'].item()
            vector_high = dil_data.loc[dil_data['concentration'] == max_dilution,
                                       'decision_vector'].item()
            decision = decision_matrix[vector_high, vector_low].item()
            if decision in [best_decision, max_dilution]:
                val = dil_data.loc[dil_data['concentration'] == decision,
                                   analyte].item()
                well = dil_data.loc[dil_data['concentration'] == decision,
                                    'well'].item()
                error = dil_data.loc[dil_data['concentration'] == decision,
                                    'error'].item()
            elif decision == 'fail':
                val = 'fail'
                well = 'fail'
                error = np.nan
                error_pids[pid] = '{} failure'.format(analyte)
            else:
                raise ValueError("Unexpected decision value: {}".format(decision))
            other_dilutions = [val for val in patient_data['concentration'].unique()]
            other_dilutions = [float(val) for val in other_dilutions if val != 'fail']
            max_dilution = int(max(other_dilutions))
            df_decision = decision if decision != 'fail' else np.nan
            best_decision = decision
            best_df = best_df.append({'patient_id': pid, 'errors': error, analyte: val,
                                      '{}_dilution'.format(analyte): df_decision,
                                      '{}_well'.format(analyte): well,
                                      '{}_max_dilution'.format(analyte): max_dilution}, ignore_index=True)
            if decision == 'fail':
                break
        patient_dfs.append(best_df)
    patient_df = pd.concat(patient_dfs)
    patient_df['errors'] = patient_df['errors'].astype('object')
    analyte_dfs.append(patient_df)
final_df = reduce(lambda left, right: pd.merge(left, right, on=['patient_id', 'errors']), analyte_dfs)

HRP2_pg_ml


C:\Users\lzoeckler\miniconda3\envs\DIAMETER\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


LDH_Pan_pg_ml
LDH_Pv_pg_ml
LDH_Pf_pg_ml
CRP_ng_ml


In [236]:
super_final = final_df.copy(deep=True)
for pid in error_pids.keys():
    error = error_pids[pid]
    pid_df = final_df.loc[final_df['patient_id'] == pid]
    pid_df['errors'] = pid_df['errors'].apply(lambda x: error if np.isnan(x) else x + ' ' + error)
    if len(pid_df) > 0:
        super_final = super_final.loc[super_final['patient_id'] != pid]
        super_final = super_final.append(pid_df)
super_final.tail()

C:\Users\lzoeckler\miniconda3\envs\DIAMETER\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,patient_id,errors,HRP2_pg_ml,HRP2_pg_ml_dilution,HRP2_pg_ml_well,HRP2_pg_ml_max_dilution,LDH_Pan_pg_ml,LDH_Pan_pg_ml_dilution,LDH_Pan_pg_ml_well,LDH_Pan_pg_ml_max_dilution,...,LDH_Pv_pg_ml_well,LDH_Pv_pg_ml_max_dilution,LDH_Pf_pg_ml,LDH_Pf_pg_ml_dilution,LDH_Pf_pg_ml_well,LDH_Pf_pg_ml_max_dilution,CRP_ng_ml,CRP_ng_ml_dilution,CRP_ng_ml_well,CRP_ng_ml_max_dilution
20,pellet_21_sample_wb,LDH_Pan_pg_ml failure,< 0.69,1,c12,400,fail,NaN,fail,400,...,d12,400,29.86,1,c12,400,< 25.86,1,c12,400
21,pellet_26_sample_wb,LDH_Pan_pg_ml failure,< 0.69,1,c11,400,fail,NaN,fail,400,...,d11,400,23.43,1,c11,400,< 25.86,1,c11,400
22,pellet_2_sample_wb,LDH_Pan_pg_ml failure,< 0.69,1,c8,400,fail,NaN,fail,400,...,d8,400,15.43,1,c8,400,< 25.86,1,c8,400
23,pellet_4_sample_wb,LDH_Pan_pg_ml failure,< 0.69,1,c9,400,fail,NaN,fail,400,...,d9,400,12.7,1,c9,400,< 25.86,1,c9,400
24,pellet_6_sample_wb,LDH_Pan_pg_ml failure,< 0.69,1,c10,400,fail,NaN,fail,400,...,d10,400,15.77,1,c10,400,< 25.86,1,c10,400


In [237]:
super_final['errors'].unique()

array([nan, 'LDH_Pan_pg_ml failure'], dtype=object)

In [238]:
final_ids = super_final['patient_id'].unique().tolist()
final_set = set(final_ids)
assert final_set == sample_set, print(sample_set - final_set)

In [239]:
super_final.to_csv('C:/Users/lzoeckler/Desktop/4plex/allison_output/final_dillutions.csv', index=False)